In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

### panoramas of cinema online

# //IMAGE SEARCH
v0.01

In [2]:
# REQUEST INDEXES
import requests
r = requests.get("https://panoramas-of-cinema.s3.eu-central-1.amazonaws.com/indexes/videotheques_inv.json")
query_response = r.json()

# 1. SEARCH VIDEOTHEQUE

### places index

In [3]:
# SHOW INDEXES 
for e in sorted(query_response['places']):
    print(e, end='  .  ')

airplane_cabin  .  airport_terminal  .  alcove  .  alley  .  amphitheater  .  amusement_arcade  .  amusement_park  .  apartment_building_outdoor  .  aquarium  .  aqueduct  .  arcade  .  arch  .  archaelogical_excavation  .  archive  .  arena_hockey  .  arena_performance  .  arena_rodeo  .  army_base  .  art_gallery  .  artists_loft  .  assembly_line  .  attic  .  auditorium  .  auto_factory  .  auto_showroom  .  badlands  .  balcony_interior  .  ball_pit  .  ballroom  .  bamboo_forest  .  bank_vault  .  banquet_hall  .  bar  .  barndoor  .  basement  .  basketball_court_indoor  .  bathroom  .  bazaar_indoor  .  bazaar_outdoor  .  beach  .  beach_house  .  beauty_salon  .  bedchamber  .  bedroom  .  beer_garden  .  beer_hall  .  berth  .  biology_laboratory  .  boardwalk  .  bookstore  .  booth_indoor  .  bow_window_indoor  .  bowling_alley  .  boxing_ring  .  bridge  .  bullring  .  burial_chamber  .  bus_interior  .  bus_station_indoor  .  butchers_shop  .  cabin_outdoor  .  campsite 

### objects index

In [4]:
# SHOW INDEXES 
for e in sorted(query_response['objects']):
    print(e, end='  .  ')

aeroplane  .  apple  .  backpack  .  banana  .  baseball_bat  .  baseball_glove  .  bear  .  bed  .  bench  .  bicycle  .  bird  .  boat  .  book  .  bottle  .  bowl  .  broccoli  .  bus  .  cake  .  car  .  carrot  .  cat  .  cell_phone  .  chair  .  clock  .  cow  .  cup  .  diningtable  .  dog  .  donut  .  elephant  .  fire_hydrant  .  fork  .  frisbee  .  giraffe  .  handbag  .  horse  .  hot_dog  .  keyboard  .  kite  .  knife  .  laptop  .  microwave  .  motorbike  .  mouse  .  orange  .  oven  .  parking_meter  .  person  .  pizza  .  pottedplant  .  refrigerator  .  remote  .  sandwich  .  scissors  .  sheep  .  sink  .  skateboard  .  sofa  .  spoon  .  sports_ball  .  stop_sign  .  suitcase  .  surfboard  .  teddy_bear  .  tennis_racket  .  tie  .  toilet  .  toothbrush  .  traffic_light  .  train  .  truck  .  tvmonitor  .  umbrella  .  vase  .  wine_glass  .  zebra  .  

### > type indexes and tune values

In [5]:
# DISPLAY WIDGETS 
from IPython.display import display, clear_output
from ipywidgets import interactive, interact
import ipywidgets as widgets

def p(place, min_conf, max_conf):
    return

def o(obj, min_conf, max_conf):
    return

w_places = interactive(p, place='hospital_room',
                min_conf=widgets.FloatSlider(min=0.00, max=1., step=.001, value=.0),
                max_conf=widgets.FloatSlider(min=0.00, max=1., step=.001, value=1.) )

w_objects = interactive(o, obj='chair',
                min_conf=widgets.FloatSlider(min=0.00, max=1., step=.001, value=.0),
                max_conf=widgets.FloatSlider(min=0.00, max=1., step=.001, value=1.) )


display(w_places)
display(w_objects)

interactive(children=(Text(value='hospital_room', description='place'), FloatSlider(value=0.0, description='mi…

interactive(children=(Text(value='chair', description='obj'), FloatSlider(value=0.0, description='min_conf', m…

### > check faces

In [6]:
def f(no_faces, small_faces):
    return
w_faces = interactive(f, no_faces=True, small_faces=True)
display(w_faces)

interactive(children=(Checkbox(value=True, description='no_faces'), Checkbox(value=True, description='small_fa…

In [7]:
# REFRESH QUERY
def refresh_query(event):
    clear_output()
    display(refresh_button)
    
    place = w_places.kwargs['place']
    place_conf = "%s,%s"%(w_places.kwargs['min_conf'],w_places.kwargs['max_conf'])

    obj = w_objects.kwargs['obj']
    obj_conf = "%s,%s"%(w_objects.kwargs['min_conf'],w_objects.kwargs['max_conf'])

    faces = str()
    if w_faces.kwargs['no_faces']:
        faces = faces+'no_faces'
    if w_faces.kwargs['small_faces']:
        if len(faces) > 0:
            faces = faces+',small_faces'
        else:
            faces = faces+'small_faces'

    # 
    this_query = 'http://pocquery-env.eba-ap3n3cr3.eu-central-1.elasticbeanstalk.com/query?'
    if place: this_query = this_query+'places=%s,%s'%(place,place_conf)
    if faces: this_query = this_query+'&faces=%s'%faces
    if obj: this_query = this_query+'&objects=%s,%s'%(obj,obj_conf)
    
    global my_frames
    my_frames = requests.get(this_query).json()['response']
    print (len(my_frames), 'images with the search : %s'%this_query[73:])
    
    global my_globals
    my_globals = {'plotted':[], 'w_buttons':[]}

                    
refresh_button = widgets.Button(description="search", button_style='info')
display(refresh_button)
refresh_button.on_click(refresh_query)

Button(button_style='info', description='search', style=ButtonStyle())

# 2. SHOW VIDEOTHEQUE RESULTS

In [8]:
# PLOT RANDOM FRAMES
def plot_frames(event):
    clear_output()
    display(plot_button)
    
    how_many = 16
    plot_x, plot_y = 4,4

    if len(my_frames) < how_many:
        how_many = len(my_frames)

    all_i = [ i for i in range(len(my_frames)) ]

    import random
    on_off = True
    this_sample = []
    while on_off:
        if len(my_globals['plotted']) == len(my_frames):
            my_globals['plotted'] = []
            print("...you've seen it all")
        r = random.choice(all_i)
        if r in my_globals['plotted']:
            pass
        else:
            this_sample.append(r)
            my_globals['plotted'].append(r)
            if len(my_globals['plotted']) == len(my_frames):
                print('...last batch')
                break
        if len(this_sample) == how_many:
            break

    # OPEN IMAGES ------------------------------
    import requests
    s3BucketName = "panoramas-of-cinema"
    region_name = 'eu-central-1'

    i=0
    frames_plot = []
    for e in my_frames:
        film = e['movieId']
        frame = e['frameId']
        if i in this_sample:
            try:
                objectKey = "frames_db%s/by_scene%s" % (film, frame)
                this_frame = "https://%s.s3.%s.amazonaws.com/%s"%(s3BucketName,region_name,objectKey)
                img = requests.get(this_frame).content          
                frames_plot.append( (film, frame, img) )
            except:
                raise 
        i+=1

    # PLOT ------------------------------
    def on_button_clicked(event):
        global movie_2p
        global frame_2p
        this_button = id(event)
        G = globals()
        all_buttons = G['my_globals']['w_buttons']
        for i, b in enumerate(all_buttons):
            this_id = id(b)
            if this_button == this_id:
                movie_2p = my_globals['w_buttons'][i].description
                frame_2p = my_globals['w_buttons'][i].tooltip

    print('...plotting %s images from %s'%(how_many, len(my_frames)))
    from ipywidgets import Button, HBox, VBox
    import ipywidgets as widgets

    k = 0
    movie_index, movie_names = 0, []
    vboxes = []
    for i in range(plot_x):
        this_col = []
        for j in range(plot_y):
            if k < len(frames_plot):
                try:
                    image_w = widgets.Image(value=frames_plot[k][2], format='png', width=250) #height=400
                    button = widgets.Button(description=frames_plot[k][0], tooltip=frames_plot[k][1])
                    my_globals['w_buttons'].append(button)

                    button.on_click( on_button_clicked )

                    this_box = [image_w, button]
                    this_col = this_col+this_box

                except:
                    raise
            k+=1

        box = VBox( this_col )
        vboxes.append(box)

    display(HBox( vboxes ))
    
plot_button = widgets.Button(description="show", button_style='info')
display(plot_button)
plot_button.on_click(plot_frames)

Button(button_style='info', description='show', style=ButtonStyle())

# 3. ADD TAGS

### > click on an image and type the user and tag

In [9]:
tag_http = 'http://poctag-env.eba-vjpstfpz.eu-central-1.elasticbeanstalk.com/tags'

In [10]:
# DISPLAY WIDGETS 
def getUser(user):
    return

def getTag(tag):
    return

w_user = interactive(getUser, user='')
w_tag = interactive(getTag, tag='')

# REFRESH QUERY
def add_tag(event):
    clear_output()
    display(w_user)
    display(w_tag)
    display(add_tag_button)
    
    this_user = w_user.kwargs['user']
    this_tag = w_tag.kwargs['tag']
    this_frame = '%s,%s'%(movie_2p,frame_2p)
    this_request = tag_http+'?action=ADD&tag=%s&user=%s&frame=%s'%(this_tag,this_user,this_frame)
    r = requests.get(this_request).json()['statusCode']
    if r == 200: print('tag added to : %s %s'%(movie_2p, frame_2p))
    else: print ('error %s'%r)

add_tag_button = widgets.Button(description="add tag", button_style='info')
add_tag_button.on_click(add_tag)

display(w_user)
display(w_tag)
display(add_tag_button)

interactive(children=(Text(value='', description='user'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Text(value='', description='tag'), Output()), _dom_classes=('widget-interact',))

Button(button_style='info', description='add tag', style=ButtonStyle())

# 4. SEARCH TAGS

### users index

In [11]:
this_request = tag_http+'?action=GET_ALL_USERS'
r = requests.get(this_request).json()['body']
for e in sorted(r):
    print(e, end='  .  ')

def get_users(event):
    clear_output()

    this_request = tag_http+'?action=GET_ALL_USERS'
    r = requests.get(this_request).json()['body']
    for e in sorted(r):
        print(e, end='  .  ')
    
    display(refresh_users_button)

        
refresh_users_button = widgets.Button(description="refresh users", button_style='info')
display(refresh_users_button)
refresh_users_button.on_click(get_users)

cowboy  .  jorge  .  katia  .  monkey2  .  monkey66  .  pig  .  sheriff  .  slash  .  

Button(button_style='info', description='refresh users', style=ButtonStyle())

### tags index

In [12]:
this_request = tag_http+'?action=GET_ALL_TAGS'
r = requests.get(this_request).json()['body']
for e in sorted(r):
    print(e, end='  .  ')

def get_tags(event):
    clear_output()

    this_request = tag_http+'?action=GET_ALL_TAGS'
    r = requests.get(this_request).json()['body']
    for e in sorted(r):
        print(e, end='  .  ')
    
    display(refresh_tags_button)

        
refresh_tags_button = widgets.Button(description="refresh tags", button_style='info')
display(refresh_tags_button)
refresh_tags_button.on_click(get_tags)

beach  .  blue  .  corridor  .  danae  .  domestic  .  ice  .  kisses  .  lynchean  .  moto on water  .  mustache  .  raining  .  sleeping  .  sleepy bear  .  smile  .  steam  .  white  .  write  .  

Button(button_style='info', description='refresh tags', style=ButtonStyle())

### > type a tag and/or user

In [13]:
# BOXES 
def getUserSearch(user_search):
    return

def getTagSearch(tag_search):
    return

w_user_search = interactive(getUserSearch, user_search='')
w_tag_search = interactive(getTagSearch, tag_search='')

# BUTTON
def search_tag(event):
    clear_output()
    display(w_user_search)
    display(w_tag_search)
    display(search_tag_button)
    
    this_user = w_user_search.kwargs['user_search']
    this_tag = w_tag_search.kwargs['tag_search']
    
    global my_frames_tag
    if len(this_user) > 0 and len(this_tag) > 0:
        this_request = tag_http+'?action=GET_FRAMES_USER_TAG&tag=%s&user=%s'%(this_tag,this_user)
        my_frames_tag = requests.get(this_request).json()
        this_length = len(my_frames_tag['body'])
        print (this_length, 'images with this search')
    elif len(this_user) > 0 and len(this_tag) == 0:
        this_request = tag_http+'?action=GET_FRAMES_USER&user=%s'%(this_user)
        my_frames_tag = requests.get(this_request).json()
        this_length = len(my_frames_tag['body'])
        print (this_length, 'images with this search')
        this_tags = set([ f['tag'] for f in my_frames_tag['body'] ])
        print ('')
        print (this_user, 'is talking about : ')
        for e in sorted(this_tags): print(e, end='  .  ')
    elif len(this_user) == 0 and len(this_tag) > 0:
        this_request = tag_http+'?action=GET_FRAMES_TAG&tag=%s'%(this_tag)
        my_frames_tag = requests.get(this_request).json()
        this_length = len(my_frames_tag['body'])
        print (this_length, 'images with this search')
        this_users = set([ f['user'] for f in my_frames_tag['body'] ])
        print ('')
        print (this_tag, 'is talked about by :')
        for e in sorted(this_users): print(e, end='  .  ')
    else:
        print ('type a user name and/or a tag')
                
    global my_globals_tag
    my_globals_tag = {'plotted':[], 'w_buttons':[]}
        
display(w_user_search)
display(w_tag_search)
search_tag_button = widgets.Button(description="search tag", button_style='info')
display(search_tag_button)
search_tag_button.on_click(search_tag)

interactive(children=(Text(value='', description='user_search'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Text(value='', description='tag_search'), Output()), _dom_classes=('widget-interact',))

Button(button_style='info', description='search tag', style=ButtonStyle())

# 5. SHOW TAG RESULTS

In [14]:
# PLOT RANDOM FRAMES 
def plot_frames_tags(event):
    clear_output()
    display(plot_button_tags)
    
    my_frames = my_frames_tag['body']
    how_many = 16
    plot_x, plot_y = 4,4

    if len(my_frames) < how_many:
        how_many = len(my_frames)

    all_i = [ i for i in range(len(my_frames)) ]

    import random
    on_off = True
    this_sample = []
    while on_off:
        if len(my_globals_tag['plotted']) == len(my_frames):
            my_globals_tag['plotted'] = []
            print("...you've seen it all")
        r = random.choice(all_i)
        if r in my_globals_tag['plotted']:
            pass
        else:
            this_sample.append(r)
            my_globals_tag['plotted'].append(r)
            if len(my_globals_tag['plotted']) == len(my_frames):
                print('...last batch')
                break
        if len(this_sample) == how_many:
            break

    # OPEN IMAGES ------------------------------
    import requests

    s3BucketName = "panoramas-of-cinema"
    region_name = 'eu-central-1'

    i=0
    frames_plot = []
    for e in my_frames:
        film = e['movie']
        frame = e['frame']
        if i in this_sample:
            try:
                objectKey = "frames_db%s/by_scene%s" % (film, frame)
                this_frame = "https://%s.s3.%s.amazonaws.com/%s"%(s3BucketName,region_name,objectKey)
                img = requests.get(this_frame).content          
                frames_plot.append( (film, frame, img) )
            except:
                raise 
        i+=1

    # PLOT ------------------------------
    def on_button_clicked(event):
        global movie_2p_tag
        global frame_2p_tag
        this_button = id(event)
        G = globals()
        all_buttons = G['my_globals_tag']['w_buttons']
        for i, b in enumerate(all_buttons):
            this_id = id(b)
            if this_button == this_id:
                movie_2p_tag = my_globals_tag['w_buttons'][i].description
                frame_2p_tag = my_globals_tag['w_buttons'][i].tooltip

    print('...plotting %s images from %s'%(how_many, len(my_frames)))
    from ipywidgets import Button, HBox, VBox
    import ipywidgets as widgets

    k = 0
    movie_index, movie_names = 0, []
    vboxes = []
    for i in range(plot_x):
        this_col = []
        for j in range(plot_y):
            if k < len(frames_plot):
                try:
                    image_w = widgets.Image(value=frames_plot[k][2], format='png', width=250) #height=400
                    button = widgets.Button(description=frames_plot[k][0], tooltip=frames_plot[k][1])
                    my_globals_tag['w_buttons'].append(button)

                    button.on_click( on_button_clicked )

                    this_box = [image_w, button]
                    this_col = this_col+this_box

                except:
                    raise
            k+=1

        box = VBox( this_col )
        vboxes.append(box)

    display(HBox( vboxes ))
    
plot_button_tags = widgets.Button(description="show", button_style='info')
display(plot_button_tags)
plot_button_tags.on_click(plot_frames_tags)

Button(button_style='info', description='show', style=ButtonStyle())

# 6. DELETE TAGS

### > click on an image

In [15]:
def delete_tag(event):
    clear_output()
    display(delete_tag_button)
        
    this_user = w_user_search.kwargs['user_search']
    this_tag = w_tag_search.kwargs['tag_search']
    if len(this_user) > 0 and len(this_tag) > 0:
        this_frame = '%s,%s'%(movie_2p_tag,frame_2p_tag)
        this_request = tag_http+'?action=DELETE&tag=%s&user=%s&frame=%s'%(this_tag,this_user,this_frame)
        r = requests.get(this_request).json()['statusCode']
        if r == 200: print('tag deleted : %s %s'%(movie_2p_tag, frame_2p_tag))
        else: print ('error %s'%r)
    else:
        print ('! user or tag is missing')

    
delete_tag_button = widgets.Button(description="delete", button_style='info')
display(delete_tag_button)
delete_tag_button.on_click(delete_tag)

Button(button_style='info', description='delete', style=ButtonStyle())